# Output evaluación compra directa
## variables
* estimacion_precio_electricidad:dict  (Estimación precios sheet)
* desglose_consumo:dict (Consumo sheet)
* evaluacion_ppa_años:dict (Rentabilidad ESCO sheet)
* O&M_active: bool
* calculo_flujo_cliente:dict   (Cliente sheet)


Librerías necesarias

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt 

In [2]:
evaluacion_compra_directa= {}

In [3]:
from pickle import TRUE


with open('../variables/desglose_consumo.json') as archivo_consumo:
    datos_consumo_json=json.load(archivo_consumo)
datos_consumo=pd.DataFrame(datos_consumo_json)


with open('../variables/estimacion_precio_electricidad.json') as archivo_estimacion:
    datos_precio_electricidad_json=json.load(archivo_estimacion)
datos_precio_electricidad=pd.DataFrame(datos_precio_electricidad_json)


with open('../variables/evaluacion_ppa_anos.json') as archivo_evaluacion:
    datos_evaluacion_ppa_anos_json=json.load(archivo_evaluacion)
datos_evaluacion_ppa_anos=pd.DataFrame(datos_evaluacion_ppa_anos_json)


with open('../variables/calculo_flujo_cliente.json') as archivo_flujo_cliente:
    datos_flujo_cliente_json=json.load(archivo_flujo_cliente)
datos_flujo_cliente=pd.DataFrame(datos_flujo_cliente_json)

o_m_admin=TRUE

In [4]:
datos_precio_electricidad

,2022,2023,2024
variacion proyectada,10.770000,10.770000,15.000000
Precio estimado energia CLP,66.960000,66.960000,NaN
Fondo estab y emergencia (cons>1000kWh/mes),3.000000,3.000000,43.481000
Precio estimado energia CLP total,2.800000,2.800000,9.500000
Precio estm. real CLP Energia,69.760000,69.760000,85.720000
Variacion proyectada transporte,0.000000,0.000000,0.000000
Precio estm. CLP Transporte,19.210000,19.210000,19.210000
Precio estm. real CLP Transporte,19.210000,19.210000,19.210000
Precio estm. nominal CLP Transporte,21.122669,21.122669,22.005496
Precio estm. nominal CLP energia,104.767248,104.767248,110.550268


In [5]:
datos_consumo

,2022,2023,2024
Consumo cliente,0,370657.0000,370657.0000
Generacion ano,0,457696.0000,457696.0000
Inyeccion,0,320387.0000,318945.0000
Autoconsumo,0,137.3090,137.3090
Consumo distribuidora,0,233348.2087,233966.0983


In [6]:
datos_evaluacion_ppa_anos

,2022,2023,2024
Ingresos y Egresos,"{'Ingreso venta energia PPA': 0, 'Ingreso pote...","{'Ingreso venta energia PPA': 40039542, 'Ingre...","{'Ingreso venta energia PPA': 42059551, 'Ingre..."
Flujos,"{'Depreciacion_positiva': 0, 'EBIT': 0, 'Gasto...","{'Depreciacion_positiva': -13458861, 'EBIT': 2...","{'Depreciacion_positiva': -13458861, 'EBIT': 2..."


In [7]:
datos_flujo_cliente

,Calculo flujo cliente
Consumo,72.1
Inyeccion,320387.1796
Margen de O&M,20
Margen de Venta adicionales,5
Margen de Venta planta solar,17
Tasa de descuento para VAN,6
Tipo de tarifa*,AT4.3
"Transporte, servicio etc.*",19.21
Venta adicionales,23157895
Venta planta solar,194585942


## Recorrer por año data json
* datos_precio_electricidad
* datos_consumo
* datos_evaluacion_ppa_anos

### VARIABLES DE USO PARA EVALUACIÓN COMPRA DIRECTA
* Estimacion precio electricidad (Estimación precios sheet)
  * Precio estm. nominal CLP energia

* Desglose consumo (Consumo sheet)
  * Inyección

* Calculo flujo cliente (Cliente sheet)
  * Venta total planta solar


* Evaluacion ppa años (Rentabilidad ESCO sheet)
  * O&M admin



In [8]:
contador=0




for fecha in datos_consumo:
    #Por fecha
    datos_precio_electricidad_fecha=datos_precio_electricidad[fecha]
    datos_evaluacion_ppa_anos_fecha=datos_evaluacion_ppa_anos[fecha]
    datos_consumo_fecha=datos_consumo[fecha]

    #Variables
    inyeccion=datos_consumo_fecha["Inyeccion"]
    precio_estimado_energia=datos_precio_electricidad_fecha["Precio estm. nominal CLP energia"]
    costo_o_m_admin=datos_evaluacion_ppa_anos_fecha["Ingresos y Egresos"]["O&M y Admin"]
    margen_o_m= datos_flujo_cliente["Calculo flujo cliente"] ["Margen de O&M"]
    margen_o_m_porcentaje=margen_o_m/100
    consumo_distribuidora= datos_consumo_fecha["Consumo distribuidora"]
    precio_estimado_transporte =datos_precio_electricidad_fecha["Precio estm. nominal CLP Transporte"]
    consumo_cliente=datos_consumo_fecha["Consumo cliente"]




    o_m_admin_fv= (costo_o_m_admin/(1-margen_o_m_porcentaje))





    #Resultados response "Ingresos y egresos con FV"
    ingreso_inyeccion_FV=round(precio_estimado_energia*inyeccion)


    #Solamente el primer año de comienzo


    pago_distribuidora= round(-1*(consumo_distribuidora * (precio_estimado_transporte + precio_estimado_energia)))


    pago_anual_energia_fv_negativo=ingreso_inyeccion_FV  + o_m_admin_fv + pago_distribuidora




    pago_anual_energia_sin_fv= round(consumo_cliente * (precio_estimado_transporte + precio_estimado_energia))
    pago_anual_energia_con_fv=round(pago_anual_energia_fv_negativo *-1)





    if contador==0:

        inversion_planta_fv=-1*(datos_flujo_cliente["Calculo flujo cliente"]["Venta total"])
        ahorro_anual= inversion_planta_fv
        ahorro_acumulado= ahorro_anual
        linea_tir=-1*(datos_flujo_cliente["Calculo flujo cliente"] ["Venta planta solar"])


    else:
        fecha_anterior= str(int(fecha) - 1)
        evaluacion_fecha_anterior=evaluacion_compra_directa[fecha_anterior]
        ahorro_acumulado_anterior= evaluacion_fecha_anterior["Ahorro acumulado"]



        # for evaluacion in evaluacion_compra_directa[fecha_anterior]:
        #     print(evaluacion)



        ahorro_anual= round(pago_anual_energia_sin_fv - pago_anual_energia_con_fv)


        #Ahorro anual + ahorro acumulado año anterior

        ahorro_acumulado= ahorro_anual + ahorro_acumulado_anterior


        inversion_planta_fv=0
        linea_tir=ingreso_inyeccion_FV + o_m_admin_fv




    evaluacion_compra_directa[fecha]=({
        "Ingresos por inyeccion FV": ingreso_inyeccion_FV,
        "Inversion planta FV": inversion_planta_fv,
        "O&M y admin FV":o_m_admin_fv,
        "Pago distribuidora": pago_distribuidora,
        "Pago anual energia con FV-": pago_anual_energia_fv_negativo,
        "Pago anual energia sin FV":pago_anual_energia_sin_fv,
        "Pago anual energia con FV+" : pago_anual_energia_con_fv,
        "Ahorro anual": ahorro_anual,
        "Ahorro acumulado": ahorro_acumulado,
        "Linea para TIR": linea_tir

    })
    # print (evaluacion_compra_directa)
    
    with open('../evaluaciones/compra_directa/evaluacion.json', 'w') as file:
        json.dump(evaluacion_compra_directa, file, indent=4)

    
    contador += 1


    








    

In [9]:

with open('../evaluaciones/compra_directa/evaluacion.json') as archivo_evaluacion:
    datos_compra_directa_json=json.load(archivo_evaluacion)
datos_compra_directa=pd.DataFrame(datos_compra_directa_json)


In [10]:
datos_compra_directa

,2022,2023,2024
Ingresos por inyeccion FV,0.0,33566064.0,35259455.0
Inversion planta FV,-217743837.0,0.0,0.0
O&M y admin FV,0.0,-4454935.0,-4454935.0
Pago distribuidora,0.0,-29376187.0,-31013555.0
Pago anual energia con FV-,0.0,-265058.0,-209035.0
Pago anual energia sin FV,0.0,46661979.0,49132722.0
Pago anual energia con FV+,0.0,265058.0,209035.0
Ahorro anual,-217743837.0,46396921.0,48923687.0
Ahorro acumulado,-217743837.0,-171346916.0,-122423229.0
Linea para TIR,-194585942.0,29111129.0,30804520.0
